A simple Chatbot using model of hugging face

In [ ]:
from transformers.utils import logging
logging.set_verbosity_error()

In [ ]:
from transformers import pipeline

In [ ]:
chatbot = pipeline(task="conversational",
                   model="facebook/blenderbot-400M-distill")

In [ ]:
user_message = '''What are some fun activities I can do in the Summer?'''

In [ ]:
from transformers import Conversation

In [ ]:
conversation = Conversation(user_message)

In [ ]:
print(conversation)

In [ ]:
conversation = chatbot(conversation)

In [ ]:
print(conversation)

In [ ]:
conversation.add_message({"role":"user",
                          "content": ''' 
                          how can i learn to code?.'''})

In [ ]:
print(conversation)

In [ ]:
print(chatbot(conversation))

In [ ]:
import gc
del conversation
gc.collect()

Transalation and Summarizing

In [ ]:
import torch

In [ ]:
translator = pipeline(task="translation",
                      model="facebook/nllb-200-distilled-600M",
                      torch_dtype = torch.bfloat16)

In [ ]:
text = """\
My puppy is adorable, \
Your kitten is cute.
Her panda is friendly.
His llama is thoughtful. \
We all have nice pets!"""

In [ ]:
text_translated = translator(text,
                             src_lang="eng_Latn",
                             tgt_lang="fra_Latn")

In [ ]:
text_translated

Free up some memory before continuing

In [ ]:
import gc
del text_translated
gc.collect()

Summarization

In [ ]:
summarizer = pipeline(task="summarization",
                        model="facebook/bart-large-cnn",
                        torch_dtype = torch.bfloat16)

In [ ]:
text = """Paris is the capital and most populous city of France, with
          an estimated population of 2,175,601 residents as of 2018,
          in an area of more than 105 square kilometres (41 square
          miles). The City of Paris is the centre and seat of
          government of the region and province of Île-de-France, or
          Paris Region, which has an estimated population of
          12,174,880, or about 18 percent of the population of France
          as of 2017."""

In [ ]:
summary = summarizer(text, min_length=10, max_length=100)

In [ ]:
summary

In [ ]:
del summarizer
gc.collect()

Sentence  Embeddings

An embedding refers to a learned representation of words or entities in a vector space, where words or entities with similar meanings are mapped to similar vectors.

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
sentences1 = ['The dog sits outside',
              'A man is playing guitar',
              'The movies are awesome']

In [ ]:
embeddings1 = model.encode(sentences1,convert_to_tensor=True)

In [ ]:
sentences2 = ['The dog plays in the garden',
              'A man plays musical instrument.',
              'The new movie is so great']

In [ ]:
embeddings2 = model.encode(sentences2,convert_to_tensor=True)

In [ ]:
print(embeddings2)

In [ ]:
from sentence_transformers import util

In [ ]:
cosine_scores = util.cos_sim(embeddings1, embeddings2)

In [ ]:
print(cosine_scores
      )

In [ ]:
for i in range(len(sentences1)):
    print('{} \t\t {} \t\t Score: {:.4f}'.format(sentences1[i], sentences2[i], cosine_scores[i][i]) )

In [ ]:
del model
gc.collect()

Zero-Shot Audio Classification

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ashraq/esc50", split='train[:10]')

In [ ]:
audio_sample  = dataset[0]

In [ ]:
audio_sample

In [ ]:
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio_sample["audio"]["array"],
             rate=audio_sample["audio"]["sampling_rate"])

Build the Audio Classification pipeline using Transformers Library

In [ ]:
zero_shot_classifier = pipeline(task="zero-shot-audio-classification",
model ="laion/clap-htsat-unfused")

In [ ]:
zero_shot_classifier.feature_extractor.sampling_rate

In [ ]:
audio_sample["audio"]["sampling_rate"]

In [ ]:
from datasets import Audio

dataset = dataset.cast_column(
    "audio",
    Audio(sampling_rate=48_000)
)

In [ ]:
audio_sample = dataset[1]

In [ ]:
audio_sample

In [ ]:
candidate_labels = ["Sound of a dog","Sound of vacuum cleaner"]

In [ ]:
zero_shot_classifier(audio_sample["audio"]["array"],candidate_labels = candidate_labels)

In [ ]:
candidate_labels = ["Sound of a child crying",
                    "Sound of vacuum cleaner",
                    "Sound of a bird singing",
                    "Sound of an airplane"]

In [ ]:
zero_shot_classifier(audio_sample["audio"]["array"],candidate_labels = candidate_labels)

In [ ]:
del zero_shot_classifier
gc.collect()

Automatic Speech Recognition

In [ ]:
from datasets import load_dataset
from transformers import pipeline

In [ ]:
dataset = load_dataset("librispeech_asr",split="train.clean.100", streaming=True, trust_remote_code = True)

In [ ]:
example = next(iter(dataset))

In [ ]:
dataset_head = dataset.take(5)
list(dataset_head)

In [ ]:
list(dataset_head)[2]

In [ ]:
example

In [ ]:
asr = pipeline(task="automatic-speech-recognition",
               model="distil-whisper/distil-small.en")

In [ ]:
asr.feature_extractor.sampling_rate

In [ ]:
example['audio']['sampling_rate']

In [ ]:
asr(example["audio"]["array"])["text"]

In [ ]:
import os 
import gradio as gr

In [ ]:
demo=gr.Blocks()

In [ ]:
def transcribe_speech(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry.")
        return ""
    output = asr(filepath)
    return output["text"]

In [ ]:
mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="microphone",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never")

In [ ]:
file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload",
                    type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                       lines=3),
    allow_flagging="never",
)

In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe, 
         file_transcribe],
        ["Transcribe Microphone",
         "Transcribe File"],
    )

demo.launch(debug=True,share=True)

In [ ]:
demo.close()

Text to Speech

In [ ]:
from transformers.utils import logging

logging.set_verbosity_error()

In [28]:
from transformers import pipeline

narrator = pipeline("text-to-speech", model="kakao-enterprise/vits-ljs")

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
text = """
Researchers at the Allen Institute for AI, \
HuggingFace, Microsoft, the University of Washington, \
Carnegie Mellon University, and the Hebrew University of \
Jerusalem developed a tool that measures atmospheric \
carbon emitted by cloud servers while training machine \
learning models. After a model’s size, the biggest variables \
were the server’s location and time of day it was active.
"""

In [ ]:
narrated_text = narrator(text)

In [ ]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(narrated_text["audio"][0],
             rate=narrated_text["sampling_rate"])